## Deploying the Trained Models

Refer to the previous notebooks for the training job and hyperparameter tuning job names.

In [ ]:
from sagemaker.tuner import HyperparameterTuner

# Replace with your hyperparameter tuning job name below
tuning_job_name="xgb-randsearch-20220412-18-10-58"

tuner = HyperparameterTuner.attach(tuning_job_name)

In [ ]:
tuner.best_training_job()

In [ ]:
predictor = tuner.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

## Send predictions

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import StringDeserializer

predictor.serializer = CSVSerializer()
predictor.deserializer = StringDeserializer()

# Use Sample Data from 'test'
predictor.predict("28,69,1.0,0,0,1,0,0,1,0,0,1")

In [ ]:
import pandas as pd

test_data = pd.read_csv('test.csv', header=None)

# Test data without labels
test_data

for index, row, in test_data.iterrows():
    test_row = ','.join([str(i) for i in row.iloc[1:]])
    prediction = predictor.predict(test_row)
    print ("Predicted {} for Passenger {}".format(
        prediction,
        test_row))
    print(f"(True value was {row.iloc[0]})")
    test_data.loc[index, 'predicted'] = int(float(prediction))

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

print(f"accuracy {accuracy_score(test_data[0], test_data['predicted'])}")
conf_matrix = confusion_matrix(test_data[0], y_pred=test_data['predicted'])
print(conf_matrix)
sns.heatmap(conf_matrix, fmt='.3g', cmap='Blues', annot=True)

### Delete Endpoint

Not to incur charges

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(endpoint_name=predictor.endpoint_name)